# Tweet generator

## Getting Data 

### Twitter API

Pros:

* Easy to implement
* Ready data

Cons:

* Limited to the $3200$ most recent tweets.
* You need to get access to your Twitter developer account

In [ ]:
import tweepy
from tqdm import tqdm

In [ ]:
cfg = {
  "twitter" :
  {
    "api_key" : "",
    "api_secret_key" : "",
    "access_token" : "",
    "access_token_secret": ""
  },

  "models":
  {
    "toxicity-analysis" : {
      "pretrained_model": ""
    },
    "sentiment-analysis":  {
      "pretrained_model": ""
    }
  }
}

In [ ]:
auth = tweepy.OAuthHandler(cfg['twitter']['api_key'], cfg['twitter']['api_secret_key'])
auth.set_access_token(cfg['twitter']['access_token'], cfg['twitter']['access_token_secret'])
api = tweepy.API(auth);

In [ ]:
user = api.get_user('matteosalvinimi')

In [ ]:
def _clean(txt):
    return " ".join(
        [
            w if not w.endswith("...") else w[:-3]
            for w in txt.split()
            if not w.startswith("http") and not w.startswith("@")
        ]
    )

In [ ]:
for tweet in tqdm(
        tweepy.Cursor(
            api.user_timeline,
            screen_name='matteosalvinimi',
            tweet_mode="extended",
            include_rts=False,
            wait_on_rate_limit=True,
        ).items(),
        total=user.statuses_count,
    ):
  if tweet:
    with open("tweets.txt", "a") as f:
      f.write(_clean(tweet.full_text) + "<|endoftext|>")

  6%|▌         | 3235/52204 [00:44<11:18, 72.16it/s]


### Scraping with SNSCRAPE


Pros:

* Easy to implement, but needs a CLI execution.
* No limit in the number of tweets
* No need for any registration

Cons:

* Not found any

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json('salvini.json', lines=True)
df

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,outlinks,tcooutlinks,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags
0,snscrape.modules.twitter.Tweet,https://twitter.com/matteosalvinimi/status/146...,2021-12-01 17:22:18+00:00,Buon lavoro e in bocca al lupo a tutti i milit...,Buon lavoro e in bocca al lupo a tutti i milit...,1466095321809838085,"{'_type': 'snscrape.modules.twitter.User', 'us...",28,9,61,2,1466095321809838085,it,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,None,None,NaN,None,NaN,None,None,None,None,None,NaN
1,snscrape.modules.twitter.Tweet,https://twitter.com/matteosalvinimi/status/146...,2021-12-01 14:16:43+00:00,"In diretta, da Roma, dall’assemblea ALIS e sta...","In diretta, da Roma, dall’assemblea ALIS e sta...",1466048619132432385,"{'_type': 'snscrape.modules.twitter.User', 'us...",31,17,95,2,1466048619132432385,it,"<a href=""https://studio.twitter.com"" rel=""nofo...",https://studio.twitter.com,Twitter Media Studio,[https://twitter.com/i/broadcasts/1OyJADzXmdWGb],[https://t.co/zIz9A8R1kk],None,NaN,None,NaN,None,None,None,None,None,NaN
2,snscrape.modules.twitter.Tweet,https://twitter.com/matteosalvinimi/status/146...,2021-12-01 12:36:38+00:00,"Approvato, finalmente e definitivamente, in Co...","Approvato, finalmente e definitivamente, in Co...",1466023434874408964,"{'_type': 'snscrape.modules.twitter.User', 'us...",175,59,370,5,1466023434874408964,it,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,None,None,NaN,None,NaN,None,None,None,None,None,NaN
3,snscrape.modules.twitter.Tweet,https://twitter.com/matteosalvinimi/status/146...,2021-12-01 12:34:36+00:00,Approvato l’emendamento della Lega che salva i...,Approvato l’emendamento della Lega che salva i...,1466022921214865413,"{'_type': 'snscrape.modules.twitter.User', 'us...",115,34,220,7,1466022921214865413,it,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,None,None,NaN,None,NaN,None,None,None,None,None,NaN
4,snscrape.modules.twitter.Tweet,https://twitter.com/matteosalvinimi/status/146...,2021-12-01 10:56:23+00:00,…riduzione delle bollette di luce e gas.\nP.s....,…riduzione delle bollette di luce e gas.\nP.s....,1465998203057315843,"{'_type': 'snscrape.modules.twitter.User', 'us...",67,17,167,1,1465998199601180680,it,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,None,None,NaN,None,1.465998e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...",None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,snscrape.modules.twitter.Tweet,https://twitter.com/matteosalvinimi/status/125...,2020-04-29 14:30:44+00:00,"Non solo Savona, tra ieri e oggi commercianti,...","Non solo Savona, tra ieri e oggi commercianti,...",1255504795370209281,"{'_type': 'snscrape.modules.twitter.User', 'us...",185,330,1096,35,1255504795370209281,it,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,None,"[{'_type': 'snscrape.modules.twitter.Video', '...",NaN,None,NaN,None,None,None,None,None,NaN
9996,snscrape.modules.twitter.Tweet,https://twitter.com/matteosalvinimi/status/125...,2020-04-29 12:46:52+00:00,#Salvini: Critiche a Conte da parte dei cittad...,#Salvini: Critiche a Conte da parte dei cittad...,1255478656488570886,"{'_type': 'snscrape.modules.twitter.User', 'us...",364,199,1153,13,1255478656488570886,it,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,None,None,NaN,None,NaN,None,None,None,None,"[Salvini, Canale21]",NaN
9997,snscrape.modules.twitter.Twee

In [ ]:
tweet_clean = list(df.content.apply(_clean))

In [ ]:
tweet_clean

['Buon lavoro e in bocca al lupo a tutti i militanti che parteciperanno ai più di cento congressi cittadini della Lega, in attesa di quelli che saranno celebrati fra gennaio e febbraio.',
 'In diretta, da Roma, dall’assemblea ALIS e stati generali del trasporto e della logistica.',
 'Approvato, finalmente e definitivamente, in Commissione il mio emendamento per aiutare genitori separati messi in crisi da situazione Covid: un aiuto fino a 800€ al mese per pagare assegno mantenimento a figli o ex coniugi, in caso di difficoltà economiche. Dalle parole ai fatti',
 'Approvato l’emendamento della Lega che salva i trasporti eccezionali: sono stati cancellati i paletti che avrebbero messo in ginocchio il settore. Orgogliosi di essere intervenuti nell’interesse di migliaia di famiglie e imprese. Dalle parole ai fatti.',
 '…riduzione delle bollette di luce e gas. P.s. Incontro sempre più imprenditori, albergatori, ristoratori o agricoltori che mi dicono: “Salvini metti un limite a questa spesa,

In [ ]:
for tweet in tweet_clean:
  with open("tweets.txt", "a") as f:
      f.write(_clean(tweet) + "<|endoftext|>")

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 13.0 MB/s 
     |████████████████████████████████| 61 kB 505 kB/s 
     |████████████████████████████████| 3.3 MB 47.1 MB/s 
     |████████████████████████████████| 895 kB 38.8 MB/s 
     |████████████████████████████████| 596 kB 38.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!nvidia-smi

Thu Dec  2 10:29:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    36W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import re
import json
from sklearn.model_selection import train_test_split
from transformers import pipeline
from transformers import AutoTokenizer, GPT2TokenizerFast, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM
from transformers import TextDataset, DataCollatorForLanguageModeling

tokenizer = GPT2TokenizerFast.from_pretrained('GroNLP/gpt2-small-italian')

train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path='tweets.txt',
          block_size=128)

data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )

Downloading:   0%|          | 0.00/475k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/280k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/135 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/959 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
model = GPT2LMHeadModel.from_pretrained('GroNLP/gpt2-small-italian').to('cuda')
model.resize_token_embeddings(len(tokenizer))
model.config

Downloading:   0%|          | 0.00/427M [00:00<?, ?B/s]

GPT2Config {
  "_name_or_path": "GroNLP/gpt2-small-italian",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 0,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 100,
      "no_repeat_ngram_size": 4,
      "num_beams": 10,
      "repetition_penalty": 10.0,
      "temperature": 2.0,
      "top_k": 

In [ ]:
training_args = TrainingArguments(
                        output_dir= 'model/', #The output directory
                        overwrite_output_dir=True, #overwrite the content of the output directory
                        num_train_epochs=5, # number of training epochs
                        per_device_train_batch_size=8, # batch size for training
                        eval_steps = 500, # Number of update steps between two evaluations.
                        save_steps=500, # after # steps model is saved
                        warmup_steps=500, # number of warmup steps for learning rate scheduler
                        )


In [ ]:
trainer = Trainer(model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset
                    )

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 3509
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2195


Step,Training Loss
500,4.060800
1000,3.511600
1500,3.261600
2000,3.132900


Saving model checkpoint to model/checkpoint-500
Configuration saved in model/checkpoint-500/config.json
Model weights saved in model/checkpoint-500/pytorch_model.bin
Saving model checkpoint to model/checkpoint-1000
Configuration saved in model/checkpoint-1000/config.json
Model weights saved in model/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to model/checkpoint-1500
Configuration saved in model/checkpoint-1500/config.json
Model weights saved in model/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to model/checkpoint-2000
Configuration saved in model/checkpoint-2000/config.json
Model weights saved in model/checkpoint-2000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2195, training_loss=3.454653687357631, metrics={'train_runtime': 1224.9351, 'train_samples_per_second': 14.323, 'train_steps_per_second': 1.792, 'total_flos': 1146092175360000.0, 'train_loss': 3.454653687357631, 'epoch': 5.0})

In [ ]:
trainer.save_model(output_dir='model/saved_model/')

Saving model checkpoint to model/saved_model/
Configuration saved in model/saved_model/config.json
Model weights saved in model/saved_model/pytorch_model.bin


In [ ]:
from transformers import pipeline

generator = pipeline('text-generation', model= 'model/saved_model/', tokenizer=tokenizer)

loading configuration file model/saved_model/config.json
Model config GPT2Config {
  "_name_or_path": "GroNLP/gpt2-small-italian",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 0,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 100,
      "no_repeat_ngram_size": 4,
      "num_beams": 10,
     

In [ ]:
print(generator('Oggi')[0]['generated_text']);

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


Oggi di Roma, tra poco sarò in diretta su Rai Uno a #quartarepubblica. Buona serata Amici😊 🔴 LIVE 👉


In [ ]:
print(generator('La lega')[0]['generated_text']);

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


La lega è in pericolo e il governo ha promesso multe fino a 15mila euro per ogni immigrato. #QuartaRepubblica


In [ ]:
print(generator('Il capitano')[0]['generated_text']);

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


Il capitano della Polizia Penitenziaria di Catania è stato ferito ieri sera a San Vito del Tronto (Latina). Evviva la solidarietà e l’orgoglio dell'Italia! 🔴 LIVE 👉


In [ ]:
print(generator('La polizia')[0]['generated_text']);

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


La polizia ha bloccato la fuga per strada, mentre le forze dell’ordine stanno bloccando l’ingresso di decine di persone in centro. È una situazione che sta causando disagi a migliaia di donne e uomini della Polizia Penitenziaria". (2/2) 🔴 LIVE 👉
